In [1]:
!git clone https://huggingface.co/EleutherAI/gpt-j-6B

Cloning into 'gpt-j-6B'...
remote: Enumerating objects: 76, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 76 (delta 36), reused 0 (delta 0)
Unpacking objects: 100% (76/76), done.


In [2]:
%cd gpt-j-6B
!git checkout float16 --force

/content/gpt-j-6B
Branch 'float16' set up to track remote branch 'float16' from 'origin'.
Switched to a new branch 'float16'


In [3]:
!rm /content/gpt-j-6B/pytorch_model.bin

In [4]:
!wget https://huggingface.co/EleutherAI/gpt-j-6B/resolve/float16/pytorch_model.bin

--2021-12-01 19:57:53--  https://huggingface.co/EleutherAI/gpt-j-6B/resolve/float16/pytorch_model.bin
Resolving huggingface.co (huggingface.co)... 34.204.221.201, 52.22.206.115, 35.173.116.130, ...
Connecting to huggingface.co (huggingface.co)|34.204.221.201|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/EleutherAI/gpt-j-6B/319b73584bcdd552fec914980a406cc2ee4f84affaf34bd8ed87fbe14bbba28e [following]
--2021-12-01 19:57:54--  https://cdn-lfs.huggingface.co/EleutherAI/gpt-j-6B/319b73584bcdd552fec914980a406cc2ee4f84affaf34bd8ed87fbe14bbba28e
Resolving cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)... 13.224.160.119, 13.224.160.107, 13.224.160.125, ...
Connecting to cdn-lfs.huggingface.co (cdn-lfs.huggingface.co)|13.224.160.119|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12106053103 (11G) [application/octet-stream]
Saving to: ‘pytorch_model.bin’

pytorch_model.bin   100%[===================>]  11.2

In [5]:
%cd /content

/content


In [6]:
!pip install transformers
!pip install pytorch-lit

     |████████████████████████████████| 3.1 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 26.9 MB/s 
     |████████████████████████████████| 3.3 MB 38.5 MB/s 
     |████████████████████████████████| 61 kB 527 kB/s 
     |████████████████████████████████| 895 kB 39.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [7]:
from pytorch_lit import prepare_params, PartialLoader
import torch

In [8]:
weights = PartialLoader("./gpt-j-6B/pytorch_model.bin")
prepare_params(weights, ".models/gpt-j-6b-lit", dtype="float16")

creating memmap ...


100%|██████████| 341/341 [04:32<00:00,  1.25it/s]


writing meta ...
done export.


In [9]:
from transformers import AutoTokenizer
from transformers.models.gptj.modeling_gptj import GPTJForCausalLM
from transformers import AutoConfig
from transformers import pipeline
from pytorch_lit import LitModule

In [10]:
tokenizer = AutoTokenizer.from_pretrained("./gpt-j-6B")
config = AutoConfig.from_pretrained("./gpt-j-6B", output_attentions=True, output_hidden_states=True)
model = LitModule.from_params(".models/gpt-j-6b-lit",
                                lambda: GPTJForCausalLM(config),
                                device="cuda")
tokens = tokenizer("hello from pytorch-lit", return_tensors="pt")
tokens = {k: tokens[k].cuda() for k in tokens}
result = model(**tokens)

In [11]:
result

CausalLMOutputWithPast([('logits',
                         tensor([[[10.9844, 10.1406,  6.5430,  ..., -6.3672, -6.5625, -6.2656],
                                  [ 6.2188,  6.3203,  3.3867,  ..., -7.6172, -7.6133, -7.6836],
                                  [10.3516,  8.6484,  7.7734,  ..., -4.5625, -4.7422, -4.6602],
                                  ...,
                                  [13.3047, 11.2109,  8.0312,  ..., -7.1250, -6.9023, -6.7617],
                                  [ 4.8320,  7.3711,  9.4766,  ..., -5.2227, -4.9727, -4.8477],
                                  [12.8828, 12.0078,  6.5508,  ..., -4.7891, -4.6523, -4.4492]]],
                                device='cuda:0')),
                        ('past_key_values',
                         ((tensor([[[[ 1.9766e+00, -1.4111e+00, -2.1523e+00,  ..., -1.3574e+00,
                                       1.6406e+00,  1.3094e-03],
                                     [ 9.9167e-01, -3.0367e-01, -8.1765e-01,  ..., -5.5566e-

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("./gpt-j-6B")
config = AutoConfig.from_pretrained("./gpt-j-6B")
model = LitModule.from_params(".models/gpt-j-6b-lit",
                                lambda: GPTJForCausalLM(config),
                                device="cuda")
generator = pipeline('text-generation', model=model.module, tokenizer=tokenizer, device=0)
result = generator("Hello I am gpt-j model and ", max_length=14, num_return_sequences=1)
print(result)